In [1]:
import torch
import torchaudio
import torchvision
from datasets import load_dataset
from IPython.display import Audio

/home/dgj335/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
device = "cuda"
class Config: pass
config = Config()
config.batch_size = 16
config.num_workers = 12
config.grad_accum_steps = 1
config.plot_update = 64
config.patience = 64
config.min_lr = 1e-7
config.max_lr = 3e-5
config.warmup_steps = 5000
config.weight_decay = 0.
config.epochs = 2000

config.length_samples = 2**20
config.channels = 2

In [3]:
train_dataset = load_dataset("danjacobellis/musdb18hq_vss",split='train')

In [9]:
L = config.length_samples
C = config.channels
crop = torchvision.transforms.RandomCrop((2,L))
def collate_fn(batch):
    B = len(batch)
    x = torch.zeros( (B, C, 2, L), dtype=torch.float32)
    i_sample = 0
    for i_sample, sample in enumerate(batch):
        audio_mix, fs = torchaudio.load(sample['audio_mix']['bytes'])
        audio_vocal, fs = torchaudio.load(sample['audio_vocal']['bytes'])
        audio_mix = audio_mix.unsqueeze(1)
        audio_vocal = audio_vocal.unsqueeze(1)
        audio = torch.cat([audio_mix,audio_vocal],dim=1)
        x[i_sample,:,:,:] = crop(audio)
    return x[:,:,0,:], x[:,:,1,:]

In [10]:
dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    num_workers=config.num_workers,
    drop_last=True,
    shuffle=True,
    collate_fn=collate_fn
)

In [11]:
x,v = next(iter(dataloader))